<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/04_VLM_setup_validacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
"""
================================================================================
FASE VLM - REQUISITO 1: SETUP Y VALIDACIÓN
================================================================================
Trabajo Fin de Máster - Evaluación Comparativa de Técnicas de Segmentación
Universidad Oberta de Catalunya (UOC)
Autor: Jesús L.
Fecha: Diciembre 2025

Objetivo:
    Establecer infraestructura para análisis VLM (Gemini Pro Vision) integrando
    resultados de segmentación previos con análisis fotográfico automatizado.

Decisiones metodológicas documentadas:
    - EXIF extraído de archivos RAW originales (metadatos fiables)
    - Análisis visual sobre imágenes editadas (coherencia con segmentación)
    - Máscara de OneFormer (mejor modelo global)
    - Formato de entrada: imagen + overlay de máscara (lado a lado)
    - Formato de salida: JSON estructurado

Requisitos:
    - Google Colab con acceso a Google Drive
    - API Key de Google AI Studio (Gemini)
    - Índice maestro y recursos de fases anteriores
================================================================================
"""

'\n================================================================================\nFASE VLM - REQUISITO 1: SETUP Y VALIDACIÓN\n================================================================================\nTrabajo Fin de Máster - Evaluación Comparativa de Técnicas de Segmentación\nUniversidad Oberta de Catalunya (UOC)\nAutor: Jesús L.\nFecha: Diciembre 2025\n\nObjetivo:\n    Establecer infraestructura para análisis VLM (Gemini Pro Vision) integrando\n    resultados de segmentación previos con análisis fotográfico automatizado.\n\nDecisiones metodológicas documentadas:\n    - EXIF extraído de archivos RAW originales (metadatos fiables)\n    - Análisis visual sobre imágenes editadas (coherencia con segmentación)\n    - Máscara de OneFormer (mejor modelo global)\n    - Formato de entrada: imagen + overlay de máscara (lado a lado)\n    - Formato de salida: JSON estructurado\n\nRequisitos:\n    - Google Colab con acceso a Google Drive\n    - API Key de Google AI Studio (Gemini)\n    - 

In [38]:
# ==============================================================================
# INSTALACIÓN Y CONFIGURACIÓN
# ==============================================================================

!pip install google-generativeai pillow numpy pandas --quiet

In [39]:
import json
import os
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any

import numpy as np
import pandas as pd
from PIL import Image
import google.generativeai as genai

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
class ConfigVLM:
    """Configuración centralizada para el pipeline VLM."""

    # Rutas base
    BASE_PATH = Path("/content/drive/MyDrive/TFM")

    # Recursos de entrada
    INDICE_MAESTRO = BASE_PATH / "3_Analisis" / "fase1_integracion" / "indice_maestro.json"
    FOTOS_EDITADAS = BASE_PATH / "0_Imagenes"
    CARACTERISTICAS_JSON = BASE_PATH / "1_Caracteristicas" / "json"
    METRICAS_CSV = BASE_PATH / "3_Analisis" / "fase2b_correlaciones" / "metricas_fusionadas.csv"

    # Salidas VLM
    OUTPUT_DIR = BASE_PATH / "3_Analisis" / "fase_vlm"
    OUTPUT_IMAGENES = OUTPUT_DIR / "imagenes_compuestas"
    OUTPUT_RESPUESTAS = OUTPUT_DIR / "respuestas_vlm"
    OUTPUT_CONSOLIDADO = OUTPUT_DIR / "consolidado"

    # Configuración del modelo
    MODELO_SEGMENTACION = "oneformer"  # Mejor modelo global
    GEMINI_MODEL = "gemini-2.5-flash"  # Alternativa: gemini-1.5-pro

    # Parámetros de generación
    GENERATION_CONFIG = {
        "temperature": 0.3,  # Baja para respuestas consistentes
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 4096,
    }

    @classmethod
    def crear_directorios(cls) -> None:
        """Crea estructura de directorios para outputs."""
        for directorio in [cls.OUTPUT_DIR, cls.OUTPUT_IMAGENES,
                          cls.OUTPUT_RESPUESTAS, cls.OUTPUT_CONSOLIDADO]:
            directorio.mkdir(parents=True, exist_ok=True)
        print(f"[OK] Directorios creados en: {cls.OUTPUT_DIR}")

    @classmethod
    def validar_rutas(cls) -> Dict[str, bool]:
        """Valida existencia de recursos necesarios."""
        validaciones = {
            "indice_maestro": cls.INDICE_MAESTRO.exists(),
            "fotos_editadas": cls.FOTOS_EDITADAS.exists(),
            "caracteristicas": cls.CARACTERISTICAS_JSON.exists(),
            "metricas_csv": cls.METRICAS_CSV.exists(),
        }

        print("\n=== Validación de Rutas ===")
        for recurso, existe in validaciones.items():
            estado = "[OK]" if existe else "[ERROR]"
            print(f"  {estado} {recurso}")

        return validaciones

In [42]:
# ==============================================================================
# CONFIGURACIÓN DE API GEMINI
# ==============================================================================

def configurar_gemini(api_key: str) -> genai.GenerativeModel:

    genai.configure(api_key=api_key)

    modelo = genai.GenerativeModel(
        model_name=ConfigVLM.GEMINI_MODEL,
        generation_config=ConfigVLM.GENERATION_CONFIG
    )

    print(f"[OK] Modelo {ConfigVLM.GEMINI_MODEL} configurado")
    return modelo

In [43]:
def test_conexion_gemini(modelo: genai.GenerativeModel) -> bool:
    """Verifica que la conexión con Gemini funciona."""
    try:
        respuesta = modelo.generate_content("Responde solo con: OK")
        if respuesta.text:
            print(f"[OK] Conexión verificada. Respuesta: {respuesta.text.strip()}")
            return True
    except Exception as e:
        print(f"[ERROR] Fallo de conexión: {e}")
        return False

In [44]:
# # 1. Configurar API Key
# API_KEY = "XXXXXX"

# # 2. Configurar modelo
# modelo_vlm = configurar_gemini(API_KEY)

# # 3. Test de conexión
# test_conexion_gemini(modelo_vlm)

In [45]:
# ==============================================================================
# CARGA DE DATOS
# ==============================================================================

def cargar_indice_maestro(ruta: Path) -> Dict:
    """Carga el índice maestro con referencias a todos los recursos."""
    with open(ruta, 'r', encoding='utf-8') as f:
        indice = json.load(f)

    fotos = {k: v for k, v in indice.items() if isinstance(v, dict) and 'foto_id' in v}

    print(f"[OK] Índice maestro cargado: {len(fotos)} fotografías")
    return fotos

def cargar_metricas_segmentacion(ruta: Path) -> pd.DataFrame:
    """Carga métricas consolidadas de Fase 2A."""
    df = pd.read_csv(ruta)
    print(f"[OK] Métricas cargadas: {len(df)} registros")
    return df

def obtener_iou_por_foto(df_metricas: pd.DataFrame,
                          modelo: str = "oneformer") -> Dict[str, float]:
    """
    Obtiene el mejor IoU de OneFormer para cada foto.

    Args:
        df_metricas: DataFrame con métricas consolidadas
        modelo: Nombre del modelo a filtrar

    Returns:
        Diccionario {codigo_foto: mejor_iou}
    """
    # Filtrar por modelo (columna 'modelo' contiene el nombre base)
    df_modelo = df_metricas[df_metricas['modelo'].str.lower().str.contains(modelo.lower())]

    # Obtener mejor IoU por foto (columna 'codigo_foto')
    iou_por_foto = df_modelo.groupby('codigo_foto')['iou'].max().to_dict()

    print(f"[OK] IoU extraído para {len(iou_por_foto)} fotos (modelo: {modelo})")
    return iou_por_foto

def obtener_exif_desde_csv(df_metricas: pd.DataFrame,
                            codigo_foto: str) -> Dict[str, Any]:
    """
    Extrae datos EXIF del CSV fusionado para una foto específica.

    Args:
        df_metricas: DataFrame con métricas fusionadas
        codigo_foto: Identificador de la foto

    Returns:
        Diccionario con datos EXIF
    """
    # Filtrar por foto (tomar primera fila, EXIF es igual para todas las configs)
    df_foto = df_metricas[df_metricas['codigo_foto'] == codigo_foto]

    if df_foto.empty:
        return {
            "apertura": "N/A", "iso": "N/A", "distancia_focal": "N/A",
            "tiempo_exposicion": "N/A", "ancho": "N/A", "alto": "N/A"
        }

    fila = df_foto.iloc[0]

    return {
        "apertura": fila.get('exif_apertura', "N/A"),
        "iso": fila.get('exif_iso', "N/A"),
        "distancia_focal": fila.get('exif_focal', "N/A"),
        "tiempo_exposicion": fila.get('exif_exposicion_seg', "N/A"),
        "ancho": fila.get('meta_ancho', "N/A"),
        "alto": fila.get('meta_alto', "N/A"),
    }

def cargar_caracteristicas_foto(ruta_json: Path) -> Dict[str, Any]:
    """Carga características extraídas del RAW original."""
    with open(ruta_json, 'r', encoding='utf-8') as f:
        datos = json.load(f)
    return datos


def extraer_exif_relevante(caracteristicas: Dict) -> Dict[str, Any]:
    """
    Extrae métricas EXIF relevantes para el análisis VLM.

    Returns:
        Diccionario con: apertura, iso, focal, dimensiones
    """
    exif = caracteristicas.get("exif", {})
    dimensiones = caracteristicas.get("dimensiones", {})

    return {
        "apertura": exif.get("apertura_fnumber", "N/A"),
        "iso": exif.get("iso", "N/A"),
        "distancia_focal": exif.get("distancia_focal", "N/A"),
        "tiempo_exposicion": exif.get("tiempo_exposicion_segundos", "N/A"),
        "ancho": dimensiones.get("ancho", "N/A"),
        "alto": dimensiones.get("alto", "N/A"),
    }

In [46]:
# ==============================================================================
# GENERACIÓN DE IMAGEN COMPUESTA
# ==============================================================================

def cargar_mascara(ruta_npz: Path) -> np.ndarray:
    """
    Carga máscara de segmentación desde archivo NPZ.

    Maneja diferentes estructuras de NPZ según el modelo.
    """
    datos = np.load(ruta_npz, allow_pickle=True)

    # Intentar diferentes claves según estructura del modelo
    claves_posibles = ['mask', 'mascara', 'segmentation', 'arr_0']

    for clave in claves_posibles:
        if clave in datos:
            mascara = datos[clave]
            # Si es 0D (escalar), extraer el array interno
            if mascara.ndim == 0:
                mascara = mascara.item()
            return mascara

    # Si no encuentra, usar primera clave disponible
    primera_clave = list(datos.keys())[0]
    return datos[primera_clave]

def redimensionar_mascara(mascara: np.ndarray,
                          tamaño_objetivo: Tuple[int, int]) -> np.ndarray:
    """
    Redimensiona máscara al tamaño de la imagen original.

    Args:
        mascara: Array 2D con la máscara binaria
        tamaño_objetivo: (ancho, alto) de la imagen destino

    Returns:
        Máscara redimensionada
    """
    mascara_pil = Image.fromarray(mascara.astype(np.uint8) * 255)
    mascara_redim = mascara_pil.resize(tamaño_objetivo, Image.NEAREST)
    return np.array(mascara_redim) > 127

def crear_overlay_mascara(imagen: Image.Image,
                          mascara: np.ndarray,
                          color: Tuple[int, int, int] = (0, 255, 0),
                          alpha: float = 0.4) -> Image.Image:
    """
    Crea overlay semitransparente de la máscara sobre la imagen.

    Args:
        imagen: Imagen PIL original
        mascara: Array booleano 2D
        color: Color RGB del overlay
        alpha: Transparencia (0-1)

    Returns:
        Imagen con overlay aplicado
    """
    # Asegurar que la imagen está en RGB
    if imagen.mode != 'RGB':
        imagen = imagen.convert('RGB')

    # Crear capa de color
    overlay = Image.new('RGB', imagen.size, color)

    # Crear máscara de transparencia
    mascara_alpha = Image.fromarray((mascara * int(255 * alpha)).astype(np.uint8))

    # Componer
    resultado = imagen.copy()
    resultado.paste(overlay, mask=mascara_alpha)

    return resultado

def crear_imagen_compuesta(imagen_original: Image.Image,
                           mascara: np.ndarray,
                           max_ancho: int = 1024) -> Image.Image:
    """
    Crea imagen compuesta: original | overlay de máscara.

    Args:
        imagen_original: Imagen PIL
        mascara: Array booleano 2D
        max_ancho: Ancho máximo por imagen (para optimizar API)

    Returns:
        Imagen compuesta lado a lado
    """
    # Redimensionar si es necesario
    ratio = min(1.0, max_ancho / imagen_original.width)
    if ratio < 1.0:
        nuevo_tamaño = (int(imagen_original.width * ratio),
                        int(imagen_original.height * ratio))
        imagen_original = imagen_original.resize(nuevo_tamaño, Image.LANCZOS)
        mascara = redimensionar_mascara(mascara, nuevo_tamaño)

    # Crear overlay
    imagen_overlay = crear_overlay_mascara(imagen_original, mascara)

    # Combinar lado a lado
    ancho_total = imagen_original.width * 2
    alto = imagen_original.height

    compuesta = Image.new('RGB', (ancho_total, alto))
    compuesta.paste(imagen_original, (0, 0))
    compuesta.paste(imagen_overlay, (imagen_original.width, 0))

    return compuesta

In [47]:
# ==============================================================================
# PROMPT ENGINEERING
# ==============================================================================

# Plantilla de prompt para análisis fotográfico
# ------------------------------------------------------------------------------
PROMPT_ANALISIS_FOTOGRAFICO = """Eres un experto en fotografía de retrato con amplia experiencia en análisis técnico y artístico.

Analiza la siguiente imagen de retrato. A la izquierda está la fotografía original, a la derecha la misma imagen con la máscara de segmentación de la persona superpuesta en verde.

CONTEXTO TÉCNICO DE CAPTURA (extraído del archivo RAW original):
- Apertura: f/{apertura}
- ISO: {iso}
- Distancia focal: {distancia_focal}mm
- Tiempo de exposición: {tiempo_exposicion}s
- Dimensiones: {ancho}x{alto} píxeles
- Calidad de segmentación (IoU): {iou:.3f}

Proporciona tu análisis en el siguiente formato JSON exacto:

{{
  "composicion": {{
    "valoracion": <número del 1 al 10>,
    "regla_tercios": "<cumple/parcial/no cumple>",
    "espacio_negativo": "<adecuado/excesivo/insuficiente>",
    "observaciones": "<texto breve>"
  }},
  "iluminacion": {{
    "valoracion": <número del 1 al 10>,
    "tipo_principal": "<natural/artificial/mixta>",
    "direccion": "<frontal/lateral/contraluz/rembrandt/otra>",
    "ratio_estimado": "<texto, ej: 2:1, 3:1>",
    "observaciones": "<texto breve>"
  }},
  "tecnica": {{
    "enfoque": "<correcto/suave/fallido>",
    "zona_enfoque": "<ojos/rostro/otro>",
    "exposicion": "<correcta/subexpuesta/sobreexpuesta>",
    "profundidad_campo": "<muy reducida/reducida/moderada/amplia>",
    "observaciones": "<texto breve>"
  }},
  "segmentacion": {{
    "calidad_percibida": "<excelente/buena/aceptable/deficiente>",
    "bordes": "<precisos/aceptables/irregulares>",
    "zonas_problematicas": "<ninguna/cabello/manos/ropa/otra>",
    "observaciones": "<texto breve>"
  }},
  "valoracion_global": {{
    "puntuacion": <número del 1 al 10>,
    "fortalezas": ["<fortaleza 1>", "<fortaleza 2>"],
    "areas_mejora": ["<mejora 1>", "<mejora 2>"]
  }},
  "recomendaciones": [
    "<recomendación concreta 1>",
    "<recomendación concreta 2>",
    "<recomendación concreta 3>"
  ]
}}

IMPORTANTE: Responde ÚNICAMENTE con el JSON, sin texto adicional antes o después."""


# Función para generar prompt con contexto
# ------------------------------------------------------------------------------
def generar_prompt(exif: Dict[str, Any], iou: float) -> str:
    """
    Genera prompt completo con contexto técnico.

    Args:
        exif: Diccionario con datos EXIF relevantes
        iou: Valor IoU de la segmentación

    Returns:
        Prompt formateado
    """
    return PROMPT_ANALISIS_FOTOGRAFICO.format(
        apertura=exif.get("apertura", "N/A"),
        iso=exif.get("iso", "N/A"),
        distancia_focal=exif.get("distancia_focal", "N/A"),
        tiempo_exposicion=exif.get("tiempo_exposicion", "N/A"),
        ancho=exif.get("ancho", "N/A"),
        alto=exif.get("alto", "N/A"),
        iou=iou if isinstance(iou, (int, float)) else 0.0
    )

In [48]:
# ==============================================================================
# LLAMADA AL VLM
# ==============================================================================

def analizar_fotografia(modelo: genai.GenerativeModel,
                        imagen_compuesta: Image.Image,
                        prompt: str) -> Dict[str, Any]:
    """
    Envía imagen al VLM y obtiene análisis estructurado.

    Args:
        modelo: Modelo Gemini configurado
        imagen_compuesta: Imagen PIL (original + overlay)
        prompt: Prompt con contexto técnico

    Returns:
        Diccionario con análisis o error
    """
    try:
        respuesta = modelo.generate_content([prompt, imagen_compuesta])

        # Extraer texto de respuesta
        texto = respuesta.text.strip()

        # Limpiar posibles marcadores de código
        if texto.startswith("```json"):
            texto = texto[7:]
        if texto.startswith("```"):
            texto = texto[3:]
        if texto.endswith("```"):
            texto = texto[:-3]
        texto = texto.strip()

        # Parsear JSON
        analisis = json.loads(texto)
        analisis["_metadata"] = {
            "timestamp": datetime.now().isoformat(),
            "modelo_vlm": ConfigVLM.GEMINI_MODEL,
            "estado": "exito"
        }

        return analisis

    except json.JSONDecodeError as e:
        return {
            "_metadata": {
                "timestamp": datetime.now().isoformat(),
                "estado": "error_json",
                "error": str(e),
                "respuesta_raw": texto if 'texto' in dir() else "N/A"
            }
        }
    except Exception as e:
        return {
            "_metadata": {
                "timestamp": datetime.now().isoformat(),
                "estado": "error",
                "error": str(e)
            }
        }

In [49]:
# ==============================================================================
# PIPELINE DE VALIDACIÓN
# ==============================================================================

def test_pipeline_completo(modelo_vlm: genai.GenerativeModel,
                           indice: Dict,
                           df_metricas: pd.DataFrame,
                           foto_test: str = None) -> Dict[str, Any]:
    """
    Ejecuta pipeline completo con una fotografía de prueba.

    Args:
        modelo_vlm: Modelo Gemini configurado
        indice: Índice maestro (fotos en nivel raíz)
        df_metricas: DataFrame con métricas
        foto_test: Código de foto específica (opcional, ej: '_DSC0023')

    Returns:
        Resultado del análisis
    """
    print("\n" + "="*60)
    print("TEST DE PIPELINE COMPLETO")
    print("="*60)

    # Seleccionar foto de prueba (índice tiene fotos directamente en raíz)
    fotos_disponibles = list(indice.keys())
    if not fotos_disponibles:
        print("[ERROR] No hay fotos en el índice maestro")
        return None

    if foto_test and foto_test in fotos_disponibles:
        codigo_foto = foto_test
    else:
        codigo_foto = fotos_disponibles[0]

    print(f"\n[1/6] Foto seleccionada: {codigo_foto}")

    # Obtener datos de la foto del índice
    datos_foto = indice[codigo_foto]

    # Cargar imagen editada desde rutas.imagen
    print("[2/6] Cargando imagen editada...")
    ruta_imagen = Path(datos_foto.get("rutas", {}).get("imagen", ""))

    if not ruta_imagen.exists():
        # Intentar ruta alternativa
        ruta_imagen = ConfigVLM.FOTOS_EDITADAS / f"{codigo_foto}.jpg"

    if not ruta_imagen.exists():
        print(f"[ERROR] No se encuentra imagen: {ruta_imagen}")
        return None

    imagen = Image.open(ruta_imagen)
    print(f"       Tamaño: {imagen.size}")

    # Cargar máscara de OneFormer (evitando configs inválidas)
    print("[3/6] Cargando máscara OneFormer...")
    modelos_disponibles = datos_foto.get("modelos_disponibles", {})

    # Buscar configuración OneFormer válida (evitar instance y t0.85)
    mascara = None
    config_usada = None
    for config_nombre, config_datos in modelos_disponibles.items():
        if "oneformer" in config_nombre.lower():
            # Evitar configs que sabemos que fallan
            if "instance" in config_nombre.lower():
                continue
            if "t0.85" in config_nombre or "t085" in config_nombre:
                continue

            ruta_mascara = Path(config_datos.get("ruta_mascara", ""))
            if ruta_mascara.exists():
                mascara = cargar_mascara(ruta_mascara)
                config_usada = config_nombre
                print(f"       Config: {config_nombre}")
                print(f"       Shape: {mascara.shape}")
                break

    if mascara is None:
        print("[ERROR] No se encontró máscara OneFormer válida")
        return None

    # Crear imagen compuesta
    print("[4/6] Generando imagen compuesta...")
    # Redimensionar máscara al tamaño de la imagen
    mascara_redim = redimensionar_mascara(mascara, imagen.size)
    imagen_compuesta = crear_imagen_compuesta(imagen, mascara_redim)
    print(f"       Tamaño compuesta: {imagen_compuesta.size}")

    # Guardar imagen compuesta para verificación
    ruta_compuesta = ConfigVLM.OUTPUT_IMAGENES / f"{codigo_foto}_compuesta.jpg"
    imagen_compuesta.save(ruta_compuesta, quality=90)
    print(f"       Guardada en: {ruta_compuesta}")

    # Obtener EXIF desde el CSV fusionado
    print("[5/6] Extrayendo datos EXIF del CSV...")
    exif = obtener_exif_desde_csv(df_metricas, codigo_foto)
    print(f"       EXIF: f/{exif['apertura']}, ISO {exif['iso']}, {exif['distancia_focal']}mm")

    # Obtener IoU de OneFormer
    iou_dict = obtener_iou_por_foto(df_metricas, modelo="oneformer")
    iou = iou_dict.get(codigo_foto, 0.0)
    print(f"       IoU OneFormer: {iou:.4f}")

    # Generar prompt y analizar
    print("[6/6] Enviando a Gemini Pro Vision...")
    prompt = generar_prompt(exif, iou)
    resultado = analizar_fotografia(modelo_vlm, imagen_compuesta, prompt)

    # Añadir metadata de la foto al resultado
    resultado["_foto"] = {
      "codigo_foto": codigo_foto,
      "config_mascara": config_usada,
      "iou": float(iou) if iou else 0.0,
      "exif": {k: (float(v) if isinstance(v, (np.floating, np.integer)) else v) for k, v in exif.items()}
    }

    # Mostrar resultado
    estado = resultado.get("_metadata", {}).get("estado", "desconocido")
    print(f"\n       Estado: {estado}")

    if estado == "exito":
        print("\n--- RESULTADO DEL ANÁLISIS ---")
        print(json.dumps(resultado, indent=2, ensure_ascii=False))

        # Guardar resultado
        ruta_resultado = ConfigVLM.OUTPUT_RESPUESTAS / f"{codigo_foto}_analisis.json"
        with open(ruta_resultado, 'w', encoding='utf-8') as f:
            json.dump(resultado, f, indent=2, ensure_ascii=False)
        print(f"\n[OK] Resultado guardado en: {ruta_resultado}")
    else:
        print(f"\n[ERROR] {resultado.get('_metadata', {}).get('error', 'Error desconocido')}")
        if '_metadata' in resultado and 'respuesta_raw' in resultado['_metadata']:
            print(f"       Respuesta raw: {resultado['_metadata']['respuesta_raw'][:500]}...")

    return resultado

In [50]:
# ==============================================================================
# EJECUCIÓN PRINCIPAL
# ==============================================================================

def main():
    """
    Ejecución principal del Requisito 1: Setup y Validación.

    Pasos:
        1. Validar rutas y recursos
        2. Configurar API Gemini
        3. Cargar datos del TFM
        4. Ejecutar test de pipeline
    """
    print("="*70)
    print("FASE VLM - REQUISITO 1: SETUP Y VALIDACIÓN")
    print("="*70)
    print(f"Timestamp: {datetime.now().isoformat()}")

    # 1. Crear directorios
    print("\n[PASO 1] Creando estructura de directorios...")
    ConfigVLM.crear_directorios()

    # 2. Validar rutas
    print("\n[PASO 2] Validando recursos...")
    validaciones = ConfigVLM.validar_rutas()

    if not all(validaciones.values()):
        print("\n[ADVERTENCIA] Algunos recursos no están disponibles.")
        print("Verifica las rutas en ConfigVLM antes de continuar.")
        return

    # 3. Configurar Gemini
    API_KEY = ""

    modelo_vlm = configurar_gemini(API_KEY)

    # 5. Cargar datos
    print("\n[PASO 5] Cargando datos del TFM...")
    indice = cargar_indice_maestro(ConfigVLM.INDICE_MAESTRO)
    df_metricas = cargar_metricas_segmentacion(ConfigVLM.METRICAS_CSV)

    # 6. Test de pipeline
    print("\n[PASO 6] Ejecutando test de pipeline...")
    resultado = test_pipeline_completo(modelo_vlm, indice, df_metricas)

    # Resumen final
    print("\n" + "="*70)
    print("RESUMEN DE VALIDACIÓN")
    print("="*70)
    if resultado and resultado.get("_metadata", {}).get("estado") == "exito":
        print("[OK] Pipeline VLM validado correctamente")
        print("[OK] Listo para procesar las 20 fotografías")
    else:
        print("[PENDIENTE] Revisar errores antes de continuar")

In [51]:
# Ejecutar si es script principal
if __name__ == "__main__":
    main()

FASE VLM - REQUISITO 1: SETUP Y VALIDACIÓN
Timestamp: 2025-12-14T22:30:55.274007

[PASO 1] Creando estructura de directorios...
[OK] Directorios creados en: /content/drive/MyDrive/TFM/3_Analisis/fase_vlm

[PASO 2] Validando recursos...

=== Validación de Rutas ===
  [OK] indice_maestro
  [OK] fotos_editadas
  [OK] caracteristicas
  [OK] metricas_csv
[OK] Modelo gemini-2.5-flash configurado

[PASO 5] Cargando datos del TFM...
[OK] Índice maestro cargado: 20 fotografías
[OK] Métricas cargadas: 2360 registros

[PASO 6] Ejecutando test de pipeline...

TEST DE PIPELINE COMPLETO

[1/6] Foto seleccionada: _DSC0023
[2/6] Cargando imagen editada...
       Tamaño: (4000, 6000)
[3/6] Cargando máscara OneFormer...
       Config: oneformer_ade20k_tiny_panoptic_t040
       Shape: (6000, 4000)
[4/6] Generando imagen compuesta...
       Tamaño compuesta: (2048, 1536)
       Guardada en: /content/drive/MyDrive/TFM/3_Analisis/fase_vlm/imagenes_compuestas/_DSC0023_compuesta.jpg
[5/6] Extrayendo datos EXI

In [52]:
# import numpy as np
# import pandas as pd
# import json
# from pathlib import Path

# # Cargar índice maestro
# with open("/content/drive/MyDrive/TFM/3_Analisis/fase1_integracion/indice_maestro.json", 'r') as f:
#     indice = json.load(f)

# fotos = {k: v for k, v in indice.items() if isinstance(v, dict) and 'foto_id' in v}
# print(f"Total fotos: {len(fotos)}")

# invalidas = []
# total_por_modelo = {}

# for idx_foto, (foto_id, datos_foto) in enumerate(fotos.items()):
#     print(f"\n[{idx_foto+1}/{len(fotos)}] Foto: {foto_id}")

#     modelos_disponibles = datos_foto.get("modelos_disponibles", {})
#     print(f"    Configuraciones: {len(modelos_disponibles)}")

#     for config_nombre, config_datos in modelos_disponibles.items():
#         ruta_mascara = config_datos.get("ruta_mascara", "")
#         if not ruta_mascara:
#             continue

#         ruta = Path(ruta_mascara)
#         if not ruta.exists():
#             continue

#         # Identificar modelo base
#         modelo = config_nombre.split("_")[0]
#         if modelo not in total_por_modelo:
#             total_por_modelo[modelo] = {"total": 0, "invalidas": 0}

#         try:
#             datos = np.load(ruta, allow_pickle=True)
#             for clave in ['mascara', 'mask', 'masks', 'arr_0']:
#                 if clave in datos:
#                     arr = datos[clave]
#                     if hasattr(arr, 'shape') and arr.ndim >= 2:
#                         pct = (arr > 0).sum() / arr.size * 100
#                         total_por_modelo[modelo]["total"] += 1

#                         if pct > 99 or pct < 1:
#                             total_por_modelo[modelo]["invalidas"] += 1
#                             invalidas.append({
#                                 'foto': foto_id,
#                                 'modelo': modelo,
#                                 'config': config_nombre,
#                                 'pct_unos': pct
#                             })
#                     break
#         except Exception as e:
#             pass

# print("\n" + "="*60)
# print("RESUMEN FINAL")
# print("="*60)

# print("\nPor modelo:")
# for modelo, datos in sorted(total_por_modelo.items()):
#     pct = datos["invalidas"] / datos["total"] * 100 if datos["total"] > 0 else 0
#     print(f"  {modelo:15} | {datos['invalidas']:4}/{datos['total']:4} inválidas ({pct:5.1f}%)")

# print(f"\nTotal máscaras inválidas: {len(invalidas)}")

# if len(invalidas) > 0:
#     df_inv = pd.DataFrame(invalidas)
#     print(f"\nTop 10 configs con más inválidas:")
#     print(df_inv.groupby('config').size().sort_values(ascending=False).head(10))